# Data Processing Notebook

This notebook processes data obtained from various sources, including the Cochrane Library, Pfizer documents, non-plain examples from the Clinical Trials API for the Pfizer documents, and files from Trial Summaries.

In [177]:
import pandas as pd
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Loading Datasets

We have four main datasets divided into training and testing sets, each containing plain (pls) and non-plain (non_pls) texts. These datasets are loaded from CSV files for further processing and analysis. The data has undergone data augmentation by paragraphs and has been processed through an API to obtain linguistic features.

- **Dataset 1 (Cochrane Library)**:
  - `data_cochrane_train_non_pls`: Non-plain language training set.
  - `data_cochrane_train_pls`: Plain language training set.
  - `data_cochrane_test_non_pls`: Non-plain language testing set.
  - `data_cochrane_test_pls`: Plain language testing set.

- **Dataset 2 (Pfizer and Clinical Trials)**:
  - `data_pfizer_train_non_pls`: Non-plain language training set.
  - `data_pfizer_train_pls`: Plain language training set.
  - `data_pfizer_test_non_pls`: Non-plain language testing set.
  - `data_pfizer_test_pls`: Plain language testing set.

- **Dataset 3 (Trial Summaries)**:
  - `data_trial_summaries_test_pls`: Plain language testing set.

In [178]:
# Load Dataset 1 (Cochrane Library) training and testing sets
data_cochrane_train_non_pls = pd.read_csv('data/data_cochrane_train_non_pls.csv')
data_cochrane_train_pls = pd.read_csv('data/data_cochrane_train_pls.csv')
data_cochrane_test_non_pls = pd.read_csv('data/data_cochrane_test_non_pls.csv')
data_cochrane_test_pls = pd.read_csv('data/data_cochrane_test_pls.csv')

# Load Dataset 2 (Pfizer and Clinical Trials) training and testing sets
data_pfizer_train_non_pls = pd.read_csv('data/data_pfizer_train_non_pls.csv')
data_pfizer_train_pls = pd.read_csv('data/data_pfizer_train_pls.csv')
data_pfizer_test_non_pls = pd.read_csv('data/data_pfizer_test_non_pls.csv')
data_pfizer_test_pls = pd.read_csv('data/data_pfizer_test_pls.csv')

# Load Dataset 3 (Trial Summaries) only test set
data_trial_summaries_test_pls = pd.read_csv('data/data_trial_summaries_test_pls.csv')


## Labeling Data

To facilitate the identification and classification of plain and non-plain texts, we add a `label` column to each dataset:
- A label of `0` is assigned to non-plain texts.
- A label of `1` is assigned to plain texts.

In [179]:
# Add a label column to Dataset 1 (Cochrane Library) training sets
data_cochrane_train_non_pls['label'] = 0
data_cochrane_train_pls['label'] = 1

# Add a label column to Dataset 2 (Pfizer and Clinical Trials) training sets
data_pfizer_train_non_pls['label'] = 0
data_pfizer_train_pls['label'] = 1

# Add a label column to Dataset 1 (Cochrane Library) testing sets
data_cochrane_test_non_pls['label'] = 0
data_cochrane_test_pls['label'] = 1

# Add a label column to Dataset 2 (Pfizer and Clinical Trials) testing sets
data_pfizer_test_non_pls['label'] = 0
data_pfizer_test_pls['label'] = 1

# Add a label column to Dataset 3 (TS)
data_trial_summaries_test_pls['label'] = 1

## Combining and Shuffling Data

To create comprehensive training and testing datasets, we concatenate the individual datasets (both plain and non-plain texts) into single DataFrames. This helps ensure that the machine learning models have a diverse and representative set of examples to learn from.

After concatenating, we shuffle the data to remove any order bias and ensure a random distribution of samples. On the other hand, the data from TS remains separated.

In [180]:
# Concatenate training data from both datasets
data_train = pd.concat([data_cochrane_train_non_pls, data_cochrane_train_pls, data_pfizer_train_non_pls, data_pfizer_train_pls])

# Concatenate testing data from both datasets
data_test = pd.concat([data_cochrane_test_non_pls, data_cochrane_test_pls, data_pfizer_test_non_pls, data_pfizer_test_pls])

# Shuffle the training data to ensure random distribution
data_train = data_train.sample(frac=1).reset_index(drop=True)

# Shuffle the testing data to ensure random distribution
data_test = data_test.sample(frac=1).reset_index(drop=True)
data_trial_summaries_test_pls = data_trial_summaries_test_pls.sample(frac=1).reset_index(drop=True)

## Dropping columns
### Dropping Zero-Value columns
We identify and remove columns that contain only zero values to streamline the datasets.

1. Identify columns with only zero values.
2. Drop these columns from the training and testing sets.

In [181]:
# Identify columns in the training data that contain only zero values
zeros = data_train.columns[(data_train == 0).all()]
print(zeros)

Index(['conjunctions'], dtype='object')


### Dropping Columns With a Small Non-Zero Percentage

We calculate the percentage of non-zero values for each column in the training dataset. This helps in understanding the distribution of values and identifying columns that might have a significant amount of zero values.

In [182]:
mask = (data_train != 0)
percent_zeros = mask.sum() / mask.count() * 100
percent_zeros.to_frame()

,0
file,100.000000
Kincaid,100.000000
ARI,100.000000
Coleman-Liau,100.000000
FleschReadingEase,100.000000
...,...
tobeverb,99.971117
auxverb,82.705123
conjunction,99.995874
nominalization,99.971117


Next, we set a threshold of 3% to identify columns with a low percentage of non-zero values. Columns below this threshold will be listed for removal.

In [183]:
# Set a threshold for the minimum acceptable percentage of non-zero values
threshold = 3

# Identify columns that fall below the threshold
columns_to_drop = percent_zeros[percent_zeros < threshold].index

# List the columns to be dropped
columns_to_drop

Index(['conjunctions', 'events', 'languages'], dtype='object')

### Dropping Redundant Columns

We identify some columns that are unnecessary since their information is already found in others.

In [184]:
# Add to columns to drop
columns_to_drop = columns_to_drop.append(pd.Index(['characters', 'sentences', 'words', 'paragraphs']))
columns_to_drop = columns_to_drop.append(zeros)

In [185]:
columns_to_drop

Index(['conjunctions', 'events', 'languages', 'characters', 'sentences',
       'words', 'paragraphs', 'conjunctions'],
      dtype='object')

### Dropping Columns With Not Enough _p_-value

Taking into account the hypothesis tests carried out, we eliminated the columns with a low p-value.

In [186]:
columns_to_drop = columns_to_drop.append(pd.Index(['money', 'facilities']))
columns_to_drop

Index(['conjunctions', 'events', 'languages', 'characters', 'sentences',
       'words', 'paragraphs', 'conjunctions', 'money', 'facilities'],
      dtype='object')

In [187]:
# Drop the columns with low non-zero percentage from the training and testing dataset
data_train = data_train.drop(columns_to_drop, axis=1)
data_test = data_test.drop(columns_to_drop, axis=1)
data_trial_summaries_test_pls = data_trial_summaries_test_pls.drop(columns_to_drop, axis=1)

Then, we define our feature set (`X`) and label set (`y`)

In [188]:
X_train = data_train.drop(['label'], axis=1)
y_train = data_train['label']

X_test = data_test.drop(['label'], axis=1)
y_test = data_test['label']

X_ts = data_trial_summaries_test_pls.drop(['label'], axis=1)
y_ts = data_trial_summaries_test_pls['label']

## Normalization of Data

In this section, we normalize the data to ensure that all features are on a comparable scale. We have identified that certain columns require a specific normalization method based on min-max normalization, while other columns will be normalized by the count of words of their respective type.

### Special Columns

The following columns require min-max normalization:

- `total_words`
- `total_sentences`
- `total_characters`
- `characters_per_word`
- `syll_per_word`
- `words_per_sentence`
- `sentences_per_paragraph`
- `type_token_ratio`
- `syllables`
- `wordtypes`
- `Kincaid`
- `ARI`
- `Coleman-Liau`
- `FleschReadingEase`
- `GunningFogIndex`
- `LIX`
- `SMOGIndex`
- `RIX`
- `DaleChallIndex`

In [189]:
# Columns that require min-max normalization
special_columns = ['total_words', 'total_sentences', 'total_characters',                 
                   'characters_per_word','syll_per_word', 'words_per_sentence', 'sentences_per_paragraph',
                   'type_token_ratio', 'syllables', 'wordtypes', 'Kincaid', 'ARI', 'Coleman-Liau', 'FleschReadingEase',
                   'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']

Columns to ignore and we don't want to normalize.

In [190]:
cols_to_ignore = ['file', 'text']

We normalize the columns not listed in the special columns by dividing their values by the `total_words` column. This ensures these features are scaled appropriately relative to the total word count.

In [191]:
# Normalize columns not listed in special columns by dividing by total_words
for column in X_train.columns:
    if column not in special_columns:
        if column in cols_to_ignore:
            continue
        X_train[column] = X_train[column] / X_train['total_words']
        X_test[column] = X_test[column] / X_test['total_words']
        X_ts[column] = X_ts[column] / X_ts['total_words']

### Min-Max Normalization for Special Columns

For columns identified as requiring min-max normalization, we scale their values to a range between 0 and 1.

In [192]:
# Initialize dictionaries to store the minimum and maximum values for each special column
maximos = {}
minimos = {}

# Perform min-max normalization for each special column
for column in special_columns:
    if column in cols_to_ignore:
        continue
    # Calculate the minimum and maximum values across training and testing datasets
    minimum = min(X_train[column].min(), X_test[column].min())
    maximum = max(X_train[column].max(), X_test[column].max())
    
    # Store the calculated minimum and maximum values
    maximos[str(column)] = maximum
    minimos[str(column)] = minimum

    # Apply min-max normalization to the training and testing datasets
    X_train[column] = (X_train[column] - minimum) / (maximum - minimum)
    X_test[column] = (X_test[column] - minimum) / (maximum - minimum)
    X_ts[column] = (X_ts[column] - minimum) / (maximum - minimum)


## Combining Normalized Data

In this section, we combine the normalized training and testing datasets back into a single DataFrame for further analysis. We also reattach the labels to the combined dataset.

### Steps:
1. Concatenate the normalized training and testing feature sets.
2. Reattach the labels to the combined dataset.

In [193]:
# Combine the normalized training and testing feature sets into a single DataFrame
data = pd.concat([X_train, X_test])

# Reattach the labels to the combined dataset
data['label'] = pd.concat([y_train, y_test])

# Reattach the labels to the combined dataset for TS dataset.
data_ts = pd.concat([X_ts])
data_ts['label'] = y_ts

In [194]:
data

,file,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,...,syllables,wordtypes,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,nominalization,label
0,10.1002-14651858.CD015374-abstract.txt_accumul...,0.038803,0.032526,0.702397,0.927470,0.034197,0.042550,0.104468,0.032130,0.112375,...,0.078719,0.316364,0.353081,0.201422,0.559242,0.028436,0.009479,0.045024,0.037915,0
1,10.1002-14651858.CD009122.pub2-pls.txt_accumul...,0.047434,0.040097,0.834493,0.906303,0.043767,0.056349,0.137562,0.045849,0.126447,...,0.053523,0.130909,0.451049,0.265734,0.590909,0.017483,0.003497,0.020979,0.034965,1
2,10.1002-14651858.CD010324.pub2-abstract.txt_ac...,0.046527,0.035122,0.827517,0.900154,0.041839,0.051758,0.131758,0.039033,0.116878,...,0.085694,0.261818,0.451031,0.288660,0.572165,0.025773,0.000000,0.043814,0.051546,0
3,10.1002-14651858.CD006570.pub3-pls.txt_accumul...,0.084960,0.070481,0.828093,0.858119,0.078735,0.080731,0.232326,0.078810,0.105282,...,0.062206,0.165455,0.397260,0.297945,0.448630,0.034247,0.003425,0.061644,0.051370,1
4,10.1002-14651858.CD006183.pub2-abstract.txt_ac...,0.040987,0.030683,0.765711,0.912487,0.039318,0.046172,0.123660,0.033457,0.107303,...,0.047260,0.147273,0.416667,0.280303,0.545455,0.011364,0.007576,0.060606,0.053030,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844,10.1002-14651858.CD004143.pub5-abstract.txt_ac...,0.051564,0.045288,0.544759,0.928763,0.050764,0.051088,0.131758,0.037361,0.118089,...,0.174947,0.492727,0.250780,0.166493,0.518210,0.020812,0.003122,0.040583,0.019771,0
11845,10.1002-14651858.CD004772.pub4-abstract.txt_ac...,0.052099,0.047246,0.580861,0.927787,0.054543,0.057943,0.150002,0.047398,0.126799,...,0.137224,0.483636,0.320652,0.205163,0.570652,0.017663,0.001359,0.035326,0.048913,0
11846,10.1002-14651858.CD004017.pub2-pls.txt_accumul...,0.096805,0.091870,0.544985,0.886678,0.091704,0.096726,0.162608,0.072491,0.078460,...,0.048114,0.170909,0.242938,0.118644,0.276836,0.031073,0.011299,0.053672,0.016949,1
11847,10.1002-14651858.CD008288.pub2-abstract.txt_ac...,0.033128,0.024017,0.732418,0.924309,0.028768,0.038355,0.090165,0.024473,0.106154,...,0.043701,0.156364,0.391635,0.235741,0.547529,0.015209,0.003802,0.038023,0.049430,0


In [195]:
data_ts

,file,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,...,syllables,wordtypes,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,nominalization,label
0,11439-MT_MED_9011_page7.txt,0.006421,0.003015,0.478285,0.977831,0.002170,0.016611,0.006829,0.006148,0.076018,...,0.037438,0.085455,0.289634,0.106707,0.460366,0.036585,0.015244,0.015244,0.060976,1
1,11547-MT_MED_9011_page2.txt,0.014099,0.007105,0.524371,0.961574,0.009581,0.011317,0.029950,0.005478,0.032918,...,0.035445,0.103636,0.235714,0.146429,0.346429,0.053571,0.010714,0.007143,0.014286,1
2,11418-MT_MED_9011_page1.txt,0.011299,0.009056,0.574574,0.968382,0.003785,0.015366,0.011660,0.006970,0.027688,...,0.000854,0.003636,0.269565,0.104348,0.330435,0.017391,0.000000,0.026087,0.008696,1
3,11022-MT_MED_9011_page2.txt,0.019441,0.013999,0.565806,0.957532,0.016986,0.018069,0.053412,0.010715,0.022720,...,0.047402,0.138182,0.243402,0.158358,0.304985,0.052786,0.014663,0.005865,0.020528,1
4,11286-MT_MED_9011_page3.txt,0.005127,0.009103,0.509978,0.992245,0.008593,0.010430,0.025849,0.005948,0.072468,...,0.006690,-0.014545,0.200000,0.111111,0.444444,0.033333,0.000000,0.011111,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,1099-MT_MED_9011_page5.txt,-0.010352,-0.010083,0.240897,1.013658,-0.008207,-0.007605,-0.026258,-0.003478,0.037211,...,0.009680,-0.007273,0.146018,0.070796,0.358407,0.008850,0.000000,0.048673,0.008850,1
1030,1000-MT_MED_9011_page8.txt,0.018734,0.015768,0.574595,0.962211,0.013225,0.019411,0.038350,0.011849,0.047907,...,0.044982,0.232727,0.245562,0.118343,0.378698,0.035503,0.014793,0.029586,0.029586,1
1031,11200-MT_MED_9011_page4.txt,0.006158,0.010250,0.511538,0.991654,0.010917,0.019218,0.033162,0.010975,0.049660,...,0.040996,0.065455,0.268041,0.123711,0.391753,0.025773,0.007732,0.007732,0.028351,1
1032,1371-MT_MED_9011_page4.txt,0.008147,0.005886,0.504281,0.977315,0.009862,0.011087,0.030559,0.005238,0.036285,...,0.036441,0.103636,0.237342,0.151899,0.357595,0.037975,0.012658,0.022152,0.025316,1


## Saving the Data

After normalizing and combining the data, we save the cleaned and processed datasets into CSV files for future use. This includes saving the training and testing datasets, as well as separating the plain and non-plain test sets.

In [196]:
# Combine the training features and labels, then save to data_train.csv
data = pd.concat([X_train, y_train], axis=1)
data.to_csv('data_clean/data_train.csv', index=False)

# Combine the testing features and labels, then save to data_test.csv
data = pd.concat([X_test, y_test], axis=1)
data.to_csv('data_clean/data_test.csv', index=False)

data_ts.to_csv('data_clean/data_ts.csv', index=False)

In [227]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

if not os.path.exists("histograms_normalized"):
    os.makedirs("histograms_normalized")

url_test = "data_clean/data_test.csv"
url_train = "data_clean/data_train.csv"

data_test = pd.read_csv(url_test)
data_train = pd.read_csv(url_train)

data = pd.concat([data_train, data_test])

data_plain = data[data['label'] == 1]
data_no_plain = data[data['label'] == 0] 

label_mapping = {0: 'Technical', 1: 'Plain'}
data['label'] = data['label'].map(label_mapping)

columns_to_plot = data.columns[1:-1]
sns.set_context("talk")

for column in columns_to_plot:
    plt.figure(figsize=(20, 12))
    
    sns.histplot(data=data_plain, x=column, bins=100, alpha=0.5, label='Plain', edgecolor=None, stat="percent")
    
    sns.histplot(data=data_no_plain, x=column, bins=100, alpha=0.5, label='Technical', edgecolor=None, stat="percent")
    
    plt.legend(loc='upper right', fontsize=32)
    plt.title(f'Variable: {column}', fontsize=32)  
    plt.xlabel("Proportion", fontsize=32)  
    plt.ylabel('Percentage of texts (%)', fontsize=32)  
    plt.xticks(fontsize=26)  
    plt.yticks(fontsize=26)  
    plt.savefig(f'histograms_percentage/png/{column}.png')
    plt.close()

In [228]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

if not os.path.exists("histograms_normalized"):
    os.makedirs("histograms_normalized")

url_test = "data_clean/data_test.csv"
url_train = "data_clean/data_train.csv"

data_test = pd.read_csv(url_test)
data_train = pd.read_csv(url_train)

data = pd.concat([data_train, data_test])

data_plain = data[data['label'] == 1]
data_no_plain = data[data['label'] == 0] 

label_mapping = {0: 'Technical', 1: 'Plain'}
data['label'] = data['label'].map(label_mapping)

columns_to_plot = data.columns[1:-1]
sns.set_context("talk")

for column in columns_to_plot:
    plt.figure(figsize=(20, 12))
    
    sns.histplot(data=data_plain, x=column, bins=100, alpha=0.5, label='Plain', edgecolor=None, stat="density")
    
    sns.histplot(data=data_no_plain, x=column, bins=100, alpha=0.5, label='Technical', edgecolor=None, stat="density")
    
    plt.legend(loc='upper right', fontsize=32)
    plt.title(f'Variable: {column}', fontsize=32)  
    plt.xlabel("Proportion", fontsize=32)  
    plt.ylabel('Density', fontsize=32)  
    plt.xticks(fontsize=26)  
    plt.yticks(fontsize=26)  
    plt.savefig(f'histograms_density/png/{column}.png')
    plt.close()